In [1]:
import re
import time
import datetime

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from data import users, transactions, campaign, engagement, churn  

# Helper Functions


In [2]:
def to_lowercase(df):
	df = df.copy()
	df.rename({i:i.lower() for i in df.columns.values}, axis=1, inplace=True)
	return df

def to_snakecase(df):
	snakecase = {i: re.sub(r"[,.;@#?!&$]+\ *", "", i.strip()).replace(" ", "_") for i in df}
	df.rename(columns=snakecase, inplace=True)
	return df

def get_Products(path="data/recodataset.csv", 
				column_mapping = {
					"fecha_dato": "report_date",
					"ncodpers": "customer_id",
					"ind_empleado": "employee_index",
					"pais_residencia": "country_residence",
					"sexo": "gender",
					"age": "age",
					"fecha_alta": "contract_start_date",
					"ind_nuevo": "new_customer_index",
					"antiguedad": "seniority_months",
					"indrel": "primary_customer_status",
					"ult_fec_cli_1t": "last_primary_customer_date",
					"indrel_1mes": "customer_type_start_month",
					"tiprel_1mes": "customer_relation_type",
					"indresi": "residence_index",
					"indext": "foreigner_index",
					"conyuemp": "spouse_employee_index",
					"canal_entrada": "join_channel",
					"indfall": "deceased_index",
					"tipodom": "address_type",
					"cod_prov": "province_code",
					"nomprov": "province_name",
					"ind_actividad_cliente": "activity_index",
					"renta": "gross_income",
					"segmento": "customer_segment",
					"ind_ahor_fin_ult1": "saving_account",
					"ind_aval_fin_ult1": "guarantee",
					"ind_cco_fin_ult1": "current_account",
					"ind_cder_fin_ult1": "derivada_account",
					"ind_cno_fin_ult1": "payroll_account",
					"ind_ctju_fin_ult1": "junior_account",
					"ind_ctma_fin_ult1": "more_particular_account",
					"ind_ctop_fin_ult1": "particular_account",
					"ind_ctpp_fin_ult1": "particular_plus_account",
					"ind_deco_fin_ult1": "short_term_deposits",
					"ind_deme_fin_ult1": "medium_term_deposits",
					"ind_dela_fin_ult1": "long_term_deposits",
					"ind_ecue_fin_ult1": "e_account",
					"ind_fond_fin_ult1": "funds",
					"ind_hip_fin_ult1": "mortgage",
					"ind_plan_fin_ult1": "pensions",
					"ind_pres_fin_ult1": "loans",
					"ind_reca_fin_ult1": "taxes",
					"ind_tjcr_fin_ult1": "credit_card",
					"ind_valo_fin_ult1": "securities",
					"ind_viv_fin_ult1": "home_account",
					"ind_nomina_ult1": "payroll",
					"ind_nom_pens_ult1": "pensions_payments",
					"ind_recibo_ult1": "direct_debit"
				}
	):
	santender = pd.read_csv(path)
	santender = santender.rename(columns=column_mapping)
	santender['customer_id'] = santender['customer_id'].apply(lambda x: 'str({}).zfill(4)'.format(x))
	santender = to_snakecase(to_lowercase(santender))
	santender['report_date'] = pd.to_datetime(santender['report_date'])
	santender['contract_start_date'] = pd.to_datetime(santender['contract_start_date'])
	santender['last_primary_customer_date'] = pd.to_datetime(santender['last_primary_customer_date'])
	str_cols = santender.select_dtypes(include='object').columns
	santender[str_cols] = santender[str_cols].apply(lambda x: x.str.strip(), axis=1)
	santender[str_cols] = santender[str_cols].replace(regex=[r'NA'], value=None)
	return santender

santender = get_Products("data/santander_train_small.csv")

# Get Data as pd.DataFrame

In [3]:
# santender.info()
# users.info()
# transactions.info()
# campaign.info()
engagement.info()
# churn.info()
# engagement

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   engagement_id     10000 non-null  int64         
 1   campaign_id       10000 non-null  object        
 2   customer_id       10000 non-null  object        
 3   engagement_date   10000 non-null  datetime64[ns]
 4   action_type       10000 non-null  object        
 5   device_type       10000 non-null  object        
 6   feedback_score    10000 non-null  int64         
 7   conversion_value  10000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 625.1+ KB


In [4]:
[print(i) for i in engagement.columns]


engagement_id
campaign_id
customer_id
engagement_date
action_type
device_type
feedback_score
conversion_value


[None, None, None, None, None, None, None, None]

# Connect to Database

In [5]:
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Mapped, mapped_column
from sqlalchemy import create_engine, MetaData, Column, Integer, String, Double, DateTime, ForeignKey
from sqlalchemy.dialects.mysql import LONGTEXT


def create_db(user="root", password="msql1234", server="localhost", database="transact"):
    SQLALCHEMY_DATABASE_URL = "mysql+pymysql://{}:{}@{}/{}".format(
        user, password, server, database
    )
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()

    return engine, SessionLocal, Base

engine, SessionLocal, Base = create_db()


# Schemas

In [6]:
Base.metadata.drop_all(bind=engine)
class Users(Base):
	__tablename__ = 'users'
	customer_id = Column(String(10), primary_key=True, nullable=False)
	name = Column(String(32))
	age = Column(Integer)
	gender = Column(String(32))
	marital_status = Column(String(32)) 
	income = Column(Double)
	employment_status = Column(String(32))
	account_type = Column(String(32))
	tenure = Column(Integer)

class Transactions(Base):
	__tablename__ = "transactions"
	transaction_id = Column(Integer, primary_key=True, autoincrement=True)
	customer_id = Column(String(10),  ForeignKey("users.customer_id", ondelete="CASCADE"), nullable=False)
	transaction_date = Column(DateTime)
	transaction_type = Column(String(32))
	amount = Column(Double)
	merchant_category = Column(String(32))
	channel = Column(String(32))

class Campaigns(Base):
	__tablename__ = "campaign"
	campaign_id = Column(String(32), primary_key=True, nullable=False)
	campaign_name = Column(String(32))
	start_date = Column(DateTime)
	end_date = Column(DateTime)
	target_segment = Column(String(32))
	budget = Column(Double)
	channel = Column(String(32))
	goal = Column(String(32))

class Engagement(Base): # Independent
	__tablename__ = "engagement"
	engagement_id = Column(Integer, primary_key=True, autoincrement=True)
	campaign_id = Column(String(32),  ForeignKey("campaign.campaign_id", ondelete="CASCADE"), nullable=False)
	customer_id = Column(String(10),  ForeignKey("users.customer_id", ondelete="CASCADE"), nullable=False)
	engagement_date = Column(DateTime)
	action_type = Column(String(32))
	device_type = Column(String(32))
	feedback_score = Column(Integer)
	conversion_value  = Column(Double)
	

class Churn(Base): # Independent
	__tablename__ = "churn"
	customer_id = Column(String(32), ForeignKey("users.customer_id", ondelete="CASCADE"), primary_key=True)
	churn_date = Column(DateTime)


class Product(Base):
	__tablename__ = 'santender'
	report_date = Column(DateTime)
	customer_id = Column(String(32), primary_key=True)
	employee_index =  Column(String(16))
	country_residence = Column(String(32))
	gender =  Column(String(16))
	age = Column(Integer)
	contract_start_date = Column(DateTime)
	new_customer_index = Column(Integer)
	seniority_months = Column(Integer)
	primary_customer_status = Column(Integer)
	last_primary_customer_date = Column(DateTime)
	customer_type_start_month = Column(Integer)
	customer_relation_type = Column(String(16))
	residence_index = Column(String(16))
	foreigner_index = Column(String(16))
	spouse_employee_index = Column(String(16))
	join_channel = Column(String(16))
	deceased_index = Column(String(16))
	address_type = Column(Integer)
	province_code = Column(Integer)
	province_name = Column(String(32))
	activity_index = Column(Integer)
	gross_income = Column(Double)
	customer_segment = Column(String(32))
	saving_account = Column(Integer)
	guarantee = Column(Integer) 
	current_account = Column(Integer)
	derivada_account = Column(Integer)
	payroll_account = Column(Integer)
	junior_account = Column(Integer)
	more_particular_account = Column(Integer)
	particular_account = Column(Integer)
	particular_plus_account = Column(Integer)
	short_term_deposits = Column(Integer)
	medium_term_deposits = Column(Integer)
	long_term_deposits = Column(Integer)
	e_account = Column(Integer)
	funds = Column(Integer)
	mortgage = Column(Integer)
	pensions = Column(Integer)
	loans = Column(Integer)
	taxes = Column(Integer)
	credit_card = Column(Integer)
	securities = Column(Integer)
	home_account = Column(Integer)
	payroll = Column(Integer)
	pensions_payments = Column(Integer)
	direct_debit = Column(Integer)



Base.metadata.create_all(engine)

# Insert into Database

In [7]:
with engine.connect() as db:
	dct = {'users': users, 'transactions':transactions,
		'churn':churn, 'campaign': campaign,
		'engagement': engagement,
		'santender': santender}
	for k,v in dct.items():
		try:
			v.to_sql(k, con=engine, if_exists='append', index=False)
			db.commit()
			print("{} Ok".format(k))
		except:
			db.rollback()
			print("{} Failed".format(k))
	db.close()

users Ok
transactions Ok
churn Ok
campaign Ok
engagement Ok
santender Ok


# Example on How to get data

In [8]:
import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine


def create_db(user="root", password="msql1234", server="localhost", database="transact"):
    SQLALCHEMY_DATABASE_URL = "mysql+pymysql://{}:{}@{}/{}".format(
        user, password, server, database
    )
    engine = create_engine(SQLALCHEMY_DATABASE_URL)

    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)
    Base = declarative_base()

    return engine, SessionLocal, Base

engine, SessionLocal, Base = create_db()

def get_data(query_string):
	with engine.connect() as db:
		fetched = pd.DataFrame(db.execute(query_string).fetchall())
		db.close()
	return fetched

# Getting Raw Churn Data

In [9]:
query_string = sqlalchemy.text(
	"""
	SELECT * 
	FROM users u
	LEFT JOIN churn c
	ON u.customer_id = c.customer_id;
	"""
)
fetched = get_data(query_string)
fetched

,customer_id,name,age,gender,marital_status,income,employment_status,account_type,tenure,customer_id,churn_date
0,0001,Customer_1,54,male,single,88032.42,part-time,business,19,None,NaT
1,0002,Customer_2,20,male,single,37781.93,part-time,savings,19,None,NaT
2,0003,Customer_3,45,female,divorced,103444.04,part-time,savings,1,None,NaT
3,0004,Customer_4,48,female,widowed,117793.09,unemployed,checking,12,None,NaT
4,0005,Customer_5,54,male,married,117631.33,self-employed,checking,17,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,Customer_1996,58,male,married,74336.73,unemployed,business,6,None,NaT
1996,1997,Customer_1997,34,male,divorced,74892.31,part-time,checking,9,None,NaT
1997,1998,Customer_1998,45,male,divorced,62575.93,full-time,business,1,None,NaT
1998,1999,Customer_1999,65,female,married,96813.84,unemployed,checking,9,1999,2024-03-07


# Getting Raw Transaction/RFM Data

In [10]:
query_string = sqlalchemy.text(
	"""
	SELECT u.*, t.amount, t.transaction_date,
	t.transaction_type, t.merchant_category, t.channel
	FROM users u, transactions t
	WHERE u.customer_id = t.customer_id;
	"""
)
fetched = get_data(query_string)
fetched

,customer_id,name,age,gender,marital_status,income,employment_status,account_type,tenure,amount,transaction_date,transaction_type,merchant_category,channel
0,0001,Customer_1,54,male,single,88032.42,part-time,business,19,1109.88,2023-05-19,deposit,dining,branch
1,0001,Customer_1,54,male,single,88032.42,part-time,business,19,265.13,2023-11-02,purchase,utilities,online
2,0001,Customer_1,54,male,single,88032.42,part-time,business,19,4983.13,2023-12-24,deposit,grocery,branch
3,0001,Customer_1,54,male,single,88032.42,part-time,business,19,1489.73,2024-09-13,deposit,clothing,branch
4,0001,Customer_1,54,male,single,88032.42,part-time,business,19,150.32,2024-02-10,withdrawal,travel,branch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39859,2000,Customer_2000,34,male,divorced,62408.84,unemployed,savings,4,331.56,2023-03-22,purchase,grocery,online
39860,2000,Customer_2000,34,male,divorced,62408.84,unemployed,savings,4,4937.15,2023-01-14,deposit,grocery,ATM
39861,2000,Customer_2000,34,male,divorced,62408.84,unemployed,savings,4,582.23,2023-09-22,withdrawal,travel,ATM
39862,2000,Customer_2000,34,male,divorced,62408.84,unemployed,savings,4,4233.49,2024-08-12,deposit,electronics,online


# Getting Raw Engagement Data

In [11]:
query_string = sqlalchemy.text(
	"""
	SELECT c.*, e.customer_id, e.engagement_date, 
	e.action_type, e.device_type, e.feedback_score,
	e.conversion_value
	FROM campaign c, engagement e
	WHERE c.campaign_id = e.campaign_id;
	"""
)
fetched = get_data(query_string)
fetched

,campaign_id,campaign_name,start_date,end_date,target_segment,budget,channel,goal,customer_id,engagement_date,action_type,device_type,feedback_score,conversion_value
0,0293,Campaign_293,2024-02-02,2024-03-02,Retirees,13577.18,app,retention,0295,2023-12-14,clicked,laptop,4,0.00
1,0017,Campaign_17,2024-10-08,2024-10-18,Retirees,12243.21,email,consideration,1299,2023-02-05,scrolled,laptop,4,0.00
2,0220,Campaign_220,2023-02-02,2023-02-11,Families,15385.17,search,retention,0832,2024-07-01,credentials,desktop,5,0.00
3,0248,Campaign_248,2024-03-02,2024-03-22,Retirees,8878.07,influencer,awareness,0843,2024-03-24,credentials,desktop,5,0.00
4,0296,Campaign_296,2024-10-09,2024-11-05,High-income,16326.20,search,consideration,0426,2024-01-12,clicked,desktop,4,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0496,Campaign_496,2024-04-07,2024-05-07,Families,15964.43,search,retention,0934,2024-04-13,scrolled,mobile,4,0.00
9996,0073,Campaign_73,2023-03-03,2023-03-12,Young Adults,16366.88,search,conversion,0227,2024-09-11,scrolled,laptop,5,0.00
9997,0082,Campaign_82,2023-08-08,2023-08-15,Families,10768.12,email,conversion,0051,2023-06-19,converted,desktop,3,727.35
9998,0325,Campaign_325,2024-07-15,2024-08-07,Families,14341.24,app,awareness,1046,2023-07-14,credentials,desktop,1,0.00


# Getting Raw Clicks/Leads Data

In [12]:
query_string = sqlalchemy.text(
	"""
	SELECT t1.campaign_id,
	SUM(t1.budget) AS mark_spent,
	t1.start_date,
	t1.channel as category,
	SUM(CASE WHEN t1.action_type = 'clicked' THEN 1 ELSE 0 END) AS clicks,
	SUM(CASE WHEN t1.action_type = 'credentials' THEN 1 ELSE 0 END) AS leads, 
	SUM(CASE WHEN t1.action_type = 'converted' THEN 1 ELSE 0 END) AS orders
	FROM 
	(SELECT c.campaign_id,
	c.campaign_name, 
	c.start_date, c.end_date,
	c.target_segment, 
	c.budget,
	c.channel,
	e.customer_id, 
	e.engagement_date, 
	e.action_type, e.device_type, e.feedback_score,
	e.conversion_value
	FROM campaign c, engagement e
	WHERE c.campaign_id = e.campaign_id) AS t1
	GROUP BY t1.campaign_id, t1.channel
	ORDER BY t1.campaign_id, t1.start_date, t1.channel;
	"""
)
fetched = get_data(query_string)
fetched

,campaign_id,mark_spent,start_date,category,clicks,leads,orders
0,0001,337092.66,2024-01-09,app,4,2,2
1,0002,203576.40,2024-05-02,influencer,3,4,2
2,0003,255036.96,2024-02-14,social,3,4,1
3,0004,322814.88,2023-09-09,app,3,1,2
4,0005,913740.08,2023-07-14,sms,4,4,3
...,...,...,...,...,...,...,...
494,0495,467128.06,2023-12-07,social,3,0,3
495,0496,351217.46,2024-04-07,search,6,3,2
496,0497,523831.56,2023-02-07,sms,5,5,1
497,0498,105732.72,2024-08-02,social,3,2,2
